<a href="https://colab.research.google.com/github/GiovanniTRA/text-to-taste/blob/main/gradio_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradio Demo: chatbot_multimodal

In [ ]:
!pip install -q gradio

In [ ]:
!pip install openai

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive/')
general_path = "/content/drive/My Drive/AriaProject_Text-to-Taste"

Mounted at /content/drive/


In [ ]:
import os
import time
import json
import random
import openai
from pprint import pprint

# OpenAI API Key
API_KEY = "<Your API key>"
os.environ["OPENAI_API_KEY"] = API_KEY

# Import OCR, recommendations and personal preferences

In [ ]:
with open(f"{general_path}/ocr_italian.txt", "rb") as image_file:
  menu = image_file.read().decode('utf-8')

In [ ]:
with open(f"{general_path}/recommendation_output_italian.txt", "rb") as image_file:
  recommendations = image_file.read().decode('utf-8')

In [ ]:
with open(f"{general_path}/photos.json", 'r') as f:
  photos = json.load(f)

# List of attributes to keep
attributes_to_keep = ["start_time", "address", "details"]

photos_less_info = []
for p in photos:
  new_p = {key: p[key] for key in attributes_to_keep }
  photos_less_info.append(new_p)

photos_str = json.dumps(photos_less_info, separators=(',', ':'))

In [ ]:
with open(f"{general_path}/Valia's preferences.txt", "rb") as image_file:
  preferences = image_file.read().decode('utf-8')

# Chatbot

In [ ]:
def chat_completion(messages: list) -> str:
    try:
        client = openai.OpenAI()

        completion = client.chat.completions.create(
            model='gpt-3.5-turbo-16k',
            # model='gpt-4',
            messages=messages,
            max_tokens=4096
        )
        return completion.choices[0].message.content
    except Exception as error:
        print("Error", error)
        return 'We are facing a technical issue at this moment.'

def generate_messages(messages: list, query: str) -> list:
    formated_messages = [
        {
            'role': 'system',
            'content': f"""You are a personal assistant and food recommender. You are given a menù, personal past food experiences in a JSON form and categories of food I like in input.
            Menù: {menu}
            Personal past food experiences in JSON format: {photos_str}
            Food categories I like: {preferences}
            Previously you recommended the following: {recommendations}"""
        }
    ]
    for m in messages:
        formated_messages.append({
            'role': 'user',
            'content': m[0]
        })
        formated_messages.append({
            'role': 'assistant',
            'content': m[1]
        })
    formated_messages.append(
        {
            'role': 'user',
            'content': query
        }
    )
    return formated_messages

def generate_response(query: str, chat_history: list) -> tuple:
  messages = generate_messages(chat_history, query)
  # pprint(messages)
  bot_message = chat_completion(messages)
  chat_history.append((query, bot_message))
  time.sleep(random.randint(0, 5))
  return '', chat_history

In [ ]:
import gradio as gr


with gr.Blocks() as demo:

    chatbot = gr.Chatbot(label='Openai Chatbot', height=550)
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    msg.submit(generate_response, [msg, chatbot], [msg, chatbot])

# demo.launch()
if __name__ == "__main__":
    demo.launch(show_api=False, debug=True)

In [ ]:
# import gradio as gr

# def greet(name):
#     return "Hello " + name + "!", "prova"

# demo = gr.Interface(fn=greet, inputs="text", outputs=["text", "text"])

# if __name__ == "__main__":
#     demo.launch(show_api=False, debug=True)